![alt text](05_kidney_metrics.png)

In [1]:
import torch
import torch.utils.data
from torch.utils.data import TensorDataset

import argparse
import time
import gc
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tqdm.notebook import tqdm

import wandb
import os

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
import h5py
import numpy as np
from loader import ICUVariableLengthLoaderTables, ICUVariableLengthDataset
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
torch.manual_seed(0)
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'NVIDIA GeForce RTX 4060 Ti')

In [4]:
import gc
gc.collect()

11

In [5]:
from net_dev import GNNStack
# from utils_todynet_binary import AverageMeter, accuracy, log_msg, get_default_train_val_test_loader

In [6]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [7]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=None, gamma=2):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma

#     def forward(self, inputs, targets):
#         ce_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-ce_loss)
#         loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
#         return loss

# class FocalLoss(nn.Module):
#     def __init__(self, alpha=None, gamma=2, num_classes=None):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         if alpha is None:
#             self.alpha = None
#         else:
#             # If alpha is a single float number, expand it to a tensor
#             if isinstance(alpha, float):
#                 assert num_classes is not None, "num_classes must be specified when alpha is a single float"
#                 self.alpha = torch.full((num_classes,), alpha)
#             else:
#                 self.alpha = alpha

#     def forward(self, inputs, targets):
#         ce_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-ce_loss)
#         if self.alpha is not None:
#             if self.alpha.type() != inputs.data.type():
#                 self.alpha = self.alpha.type_as(inputs.data)
#             at = self.alpha[targets]
#         else:
#             at = 1.0
#         loss = (at * (1 - pt) ** self.gamma * ce_loss).mean()
#         return loss

## aruguments

In [8]:
args = {
    'arch': 'dyGIN2d', #what other models I can put here?? dyGCN2d, dyGIN2d
    'dataset': 'Kidney_Reg_attentionMSE', # "AtrialFibrillation" # 'Mortality', # 'MIMIC3' Resp_failure
    'num_layers': 2,  # the number of GNN layers  3
    'groups': 32,  # the number of time series groups (num_graphs)
    'pool_ratio': 0.0,  # the ratio of pooling for nodes # initially 0.1 but changed to 0 because the node number was decreasing
    'kern_size': [3,3],  # list of time conv kernel size for each layer [9,5,3]
    'in_dim': 64,  # input dimensions of GNN stacks
    'hidden_dim': 64,  # hidden dimensions of GNN stacks
    'out_dim': 64,  # output dimensions of GNN stacks
    'workers': 0,  # number of data loading workers
    'epochs': 20,  # number of total epochs to run
    'batch_size': 8,  # mini-batch size, this is the total batch size of all GPUs
    'val_batch_size': 8,  # validation batch size
    'lr': 0.0002,  # initial learning rate
    'weight_decay': 1e-4,  # weight decay
    'evaluate': False,  # evaluate model on validation set
    'seed': 3,  # seed for initializing training
    'gpu': 0,  # GPU id to use
    'use_benchmark': True,  # use benchmark
    'tag': 'date',  # the tag for identifying the log and model files
    'loss':'bce',
    'resample':'5min',
    'series_length':2016,
}

# resample works: 5min, 15 min, 35 min, 45min. analysis will have 5,15,45

### GPU 2 is used to training in sheet 2

### TODO
## Molly douglas has filtered 116 features from 231, ~50% reduction

In [9]:
# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="mortality",
    
#     # track hyperparameters and run metadata
#     config=args
# )

In [10]:
# train_dataset = TensorDataset(data_train, label_train)
# val_dataset   = TensorDataset(data_val, label_val)

In [11]:
# train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=args['batch_size'],shuffle=True, num_workers=args['workers'], pin_memory=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args['val_batch_size'], shuffle=False,num_workers=args['workers'],pin_memory=True)

In [12]:
# def main():
#     # args = parser.parse_args()
    
#     # args.kern_size = [ int(l) for l in args.kern_size.split(",") ]

#     # if args.seed is not None:
#     random.seed(args['seed'])
#     torch.manual_seed(args['seed'])

#     main_work(args)

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


# def accuracy(output, target, topk=(1,)):
#     """Computes the accuracy over the k top predictions for the specified values of k"""
    
    
#     with torch.no_grad():
#         maxk = max(topk)
#         batch_size = target.size(0)

#         _, pred = output.topk(maxk, 1, True, True)
#         pred = pred.t()
              
#         correct = pred.eq(target.view(1, -1).expand_as(pred))
        
#         # print(output, target)

#         res = []
#         for k in topk:
#             correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
#             try:
#                 res.append(correct_k.mul_(100.0 / batch_size))
#             except:
#                 res.append(0)

#         return res

def log_msg(message, log_file):
    with open(log_file, 'a') as f:
        print(message, file=f)


def get_default_train_val_test_loader(args):

    # get dataset-id
    dsid = args['dataset']
    h5_path= 'h5_folder/ml_stage_12h.h5'
    # get dataset from .pt
    # data_train  = torch.load(f'data/UCR/{dsid}/X_train.pt')
    # data_val    = torch.load(f'data/UCR/{dsid}/X_valid.pt')
    # data_val    = torch.load(f'data/UCR/{dsid}/X_test.pt')
    
    # label_train = torch.load(f'data/UCR/{dsid}/y_train.pt')
    # label_val   = torch.load(f'data/UCR/{dsid}/y_valid.pt')
    # label_val   = torch.load(f'data/UCR/{dsid}/y_test.pt')

    # label_train = label_train.flatten().to(dtype=torch.int64)
    # label_val   = label_val.flatten().to(dtype=torch.int64)
    # init [num_variables, seq_length, num_classes]
    
    num_nodes = 231
    seq_length = 2016 #288 #2016 ### now we will reduce the sequence length to see the performance in short-range
    num_classes = 1


    # convert data & labels to TensorDataset
    # train_dataset = TensorDataset(data_train, label_train)
    # val_dataset = TensorDataset(data_val, label_val)
  
    task   = 'Dynamic_UrineOutput_2Hours_Reg' #
    maxlen            = 2016                                                                                   #|patients|  #-1 =8065  #2016     #difference | mean -1   | median -1  | mean 2016   | median 2016|
    data_loader_train = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='train') #|  1364  |  (+)25665  (+)18082  (+)7583      | 1.8155506 | 1.3075098  | 1.7475992   | 1.1815174  |
    data_loader_val   = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='val')   #|  295   |  (+)5652   (+)3967   (+)1685      | 1.7644166 | 1.2061048  | 1.8004797   | 1.2085757  |
    data_loader_test  = ICUVariableLengthDataset(source_path=h5_path, maxlen=maxlen, task=task, split='test')  #|  311   |  (+)4563   (+)3593   (+)970       | 1.7855672 | 1.2110261  | 1.7289896   | 1.1704105  |
    
    
    train_loader = DataLoader(data_loader_train, batch_size=4, shuffle=True, num_workers=1,pin_memory=True, prefetch_factor=2)
    val_loader   = DataLoader(data_loader_val,   batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)
    # val_loader  = DataLoader(data_loader_test,  batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)
    test_loader  = DataLoader(data_loader_test,  batch_size=4, shuffle=False, num_workers=1,pin_memory=True, prefetch_factor=2)


    # data_loader
    # train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=args['batch_size'],shuffle=True,num_workers=args['workers'], pin_memory=True)
    # val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=args['val_batch_size'],shuffle=False,num_workers=args['workers'],pin_memory=True)


    return train_loader, val_loader, test_loader, num_nodes, seq_length, num_classes

In [14]:
def main_work(args):
    
    random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    
    
    # init metrics
    best_mae = 0
    best_mse  = 0
    best_r2s  = 0
    
    best_test_mae = 0
    best_test_mse  = 0
    best_test_r2s   = 0
    
    if args['tag'] == 'date':
        local_date = time.strftime('%m.%d %H:%M', time.localtime(time.time()))
        args['tag'] = local_date
        
    # Use the 'tag' which now contains either the date or a custom tag along with the dataset name for the directory
    run_dir_name = f"{args['dataset']}_{args['tag']}"
   
    # Base directory for saving models
    base_model_save_dir = "saved_models"
   
    # Specific directory for this run
    specific_model_save_dir = os.path.join(base_model_save_dir, run_dir_name)
    os.makedirs(specific_model_save_dir, exist_ok=True)

    print(f"Models will be saved in: {specific_model_save_dir}")        


    log_file = 'log/{}_gpu{}_{}_{}_exp.txt'.format(args['tag'], args['gpu'], args['arch'], args['dataset'])
    
    
    if args['gpu'] is not None:
        print("Use GPU: {} for training".format(args['gpu']))


    # dataset
    train_loader, val_loader, test_loader, num_nodes, seq_length, num_classes = get_default_train_val_test_loader(args)
    
    print('features / nodes', num_nodes,'total time graphs',args['groups'],'time series length',seq_length,'classes', num_classes)
    
    # training model from net.py
    model = GNNStack(gnn_model_type=args['arch'], num_layers=args['num_layers'], 
                     groups=args['groups'], pool_ratio=args['pool_ratio'], kern_size=args['kern_size'], 
                     in_dim=args['in_dim'], hidden_dim=args['hidden_dim'], out_dim=args['out_dim'], 
                     seq_len=seq_length, num_nodes=num_nodes, num_classes=num_classes)

    # print & log
    log_msg('epochs {}, lr {}, weight_decay {}'.format(args['epochs'], args['lr'], args['weight_decay']), log_file)
    
    log_msg(str(args), log_file)


    # determine whether GPU or not
    if not torch.cuda.is_available():
        print("Warning! Using CPU!!!")
    elif args['gpu'] is not None:
        torch.cuda.set_device(args['gpu'])

        # collect cache
        gc.collect()
        torch.cuda.empty_cache()

        model = model.cuda(args['gpu'])
        if args['use_benchmark']:
            cudnn.benchmark = True
        print('Using cudnn.benchmark.')
    else:
        print("Error! We only have one gpu!!!")

        
    criterion = nn.MSELoss().cuda(args['gpu'])
       
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

    # validation
    if args['evaluate']:
        validate(val_loader, model, criterion, args)
        return

    # train & valid
    print('****************************************************')
    print('Dataset: ', args['dataset'])

    # dataset_time = AverageMeter('Time', ':6.3f')

    loss_train = []
    loss_val = []
    epoches = []
    
    ########################
    loss_test = []
    #######################
    
    ###### more lists to have values
    mae_train = []
    mse_train  = []
    r2s_train  = []
    
    mae_val   = []
    mse_val    = []
    r2s_val    = []
    ##################################################
    mae_test   = []
    mse_test    = []
    r2s_test    = []  
    #################################################
    end = time.time()
    for epoch in tqdm(range(args['epochs'])):
    # for epoch in range(args['epochs']):
        epoches += [epoch]

        # train for one epoch
        loss_train_per, output_train_per, target_train_per = train(train_loader, model, criterion, optimizer, lr_scheduler, args)
        
        loss_train += [loss_train_per]
        # calculate metric
        mae_value_train = mean_absolute_error(target_train_per, output_train_per)
        mse_value_train = mean_squared_error(target_train_per, output_train_per)
        r2s_value_train = r2_score(target_train_per, output_train_per)
        
        #new code
        mae_train  += [mae_value_train]
        mse_train  += [mse_value_train]
        r2s_train  += [r2s_value_train]

        
        msg = f'TRAIN, epoch {epoch}, train_loss {loss_train_per}, train_MAE {mae_value_train:.5f}, train_MSE {mse_value_train:.5f}, train_R2S {r2s_value_train:.5f}'

        print(f'TRAIN, epoch {epoch}, train_loss {loss_train_per:.5f}, train_MAE {mae_value_train:.5f}, train_MSE {mse_value_train:.5f}, train_R2S {r2s_value_train:.5f}')
        # tqdm.write(f'TRAIN, epoch {epoch}, train_loss {loss_train_per:.5f}, train_roc {auc_roc_value_train:.5f}, train_pr {auc_pr_value_train:.5f}, train_f1 {f1_value_train:.5f}, train_mcc {mcc_value_train:.5f}')
        log_msg(msg, log_file)

        
        # evaluate on validation set -- ####################################################################################
        loss_val_per, output_val_per, target_val_per = validate(val_loader, model, criterion, args)

        loss_val += [loss_val_per]
        #calculate metric
        # calculate metric
        # print(len(target_val_per),len(output_val_per))
        mae_value_val = mean_absolute_error(target_val_per, output_val_per)
        mse_value_val = mean_squared_error(target_val_per, output_val_per)
        r2s_value_val = r2_score(target_val_per, output_val_per)

        
        #new code

        msg = f'VAL, epoch {epoch}, val_loss {loss_val_per}, val_MAE {mae_value_val:.5f}, val_MSE {mse_value_val:.5f}, val_R2S {r2s_value_val:.5f}'
        
        print(f'VAL, epoch {epoch}, val_loss {loss_val_per:.5f}, val_MAE {mae_value_val:.5f}, val_MSE {mse_value_val:.5f}, val_R2S {r2s_value_val:.5f}')
        # tqdm.write(f'VAL, epoch {epoch}, val_loss {loss_val_per:.5f}, val_roc {auc_roc_value_val:.5f}, val_pr {auc_pr_value_val:.5f},val_f1 {f1_value_val:.5f}, val_mcc {mcc_value_val:.5f}')
        log_msg(msg, log_file)
        #########################################################################################################################
        
        # evaluate on test set
        loss_test_per, output_test_per, target_test_per = validate(test_loader, model, criterion, args)

        loss_test  += [loss_test_per]
        #calculate metric
        # calculate metric
        # print(len(target_val_per),len(output_val_per))
        mae_value_test  = mean_absolute_error(target_test_per, output_test_per)
        mse_value_test  = mean_squared_error(target_test_per, output_test_per)
        r2s_value_test  = r2_score(target_test_per, output_test_per)
        #new code

        msg = f'TEST, epoch {epoch},test_loss {loss_test_per}, test_MAE {mae_value_test:.5f},test_MSE {mse_value_test:.5f},test_R2S {r2s_value_test:.5f}'
        print(f'TEST, epoch {epoch}, test_loss {loss_test_per:.5f}, test_MAE {mae_value_test:.5f}, test_MSE {mse_value_test:.5f}, test_R2S {r2s_value_test:.5f}')
        # tqdm.write(f'TEST, epoch {epoch}, test_loss {loss_test_per:.5f}, test_roc {auc_roc_value_test:.5f}, test_pr {auc_pr_value_test:.5f}, test_f1 {f1_value_test:.5f}, test_mcc {mcc_value_test:.5f}')
        log_msg(msg, log_file)        
        
        #########################################################################################################################
        # remember best reg metrics
        best_mae  = max(mae_value_val, best_mae)
        best_mse  = max(mse_value_val, best_mse)
        best_r2s  = max(r2s_value_val, best_r2s)
        
        #########################################################################################################################
        
        best_test_mae  = max(mae_value_test, best_test_mae)
        best_test_mse  = max(mse_value_test, best_test_mse)
        best_test_r2s  = max(r2s_value_test, best_test_r2s)
 

    #     wandb.log({"train_loss": loss_train_per, "train_roc": auc_roc_value_train, "train_pr": auc_pr_value_train, \
    #                "val_loss": loss_val_per, "val_roc": auc_roc_value_val, "val_pr": auc_pr_value_val, "best_val_roc": best_roc, "best_val_pr": best_pr})
    # wandb.finish()
    
    
    
        # Construct the filename with metrics
        mae_value_val_scalar      = mae_value_val.item() if isinstance(mae_value_val, torch.Tensor) else mae_value_val
        mse_value_val_scalar      = mse_value_val.item() if isinstance(mse_value_val, torch.Tensor) else mse_value_val
        r2s_value_val_scalar      = r2s_value_val.item() if isinstance(r2s_value_val, torch.Tensor) else r2s_value_val
        #########################################################################################################################
        mae_value_test_scalar     = mae_value_test.item() if isinstance(mae_value_test, torch.Tensor) else mae_value_test
        mse_value_test_scalar     = mse_value_test.item() if isinstance(mse_value_test, torch.Tensor) else mse_value_test
        r2s_value_test_scalar     = r2s_value_test.item() if isinstance(r2s_value_test, torch.Tensor) else r2s_value_test
        #########################################################################################################################
        # Now use these scalar values in the filename
        filename = f"model_epoch_{epoch}.pth"
       
        # Continue with the existing logic to save the model
        model_path = os.path.join(specific_model_save_dir, filename)
        torch.save(model.state_dict(), model_path)

    # measure elapsed time
    dataset_time.update(time.time() - end)

    # log & print the best_acc
    msg = f'\n\n * BEST_MAE: {best_mae}\n * TIME: {dataset_time}\n'
    log_msg(msg, log_file)

    print(f' * best_MAE: {best_mae}, best_MSE: {best_mse}, best_R2S: {best_r2s}')
    print(f' * best_test_MAE: {best_test_mae}, best_test_MSE: {best_test_mse}, best_test_R2S: {best_test_r2s}')
    print(f' * time: {dataset_time}')
    print('****************************************************')


    # collect cache
    gc.collect()
    torch.cuda.empty_cache()

In [15]:
def train(train_loader, model, criterion, optimizer, lr_scheduler, args):
    losses = AverageMeter('Loss', ':.4e')
    # top1 = AverageMeter('Acc', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    
    output_list = []
    target_list = [] 

    # switch to train mode
    model.train()

    for count, (data, label, mask) in enumerate(train_loader):

        # data in cuda
        data = data.cuda(args['gpu']).type(torch.float)
        data = data.view(data.size(0), 1, data.size(2), data.size(1))
        mask = mask.cuda(args['gpu']).type(torch.bool)
        label = label.cuda(args['gpu']).type(torch.float) ### although its a regression value but its called label following the source code from classification

        # compute output
        output = model(data)
        # print(len(output))
        # print('output', output.shape, 'mask', mask.shape)
        out_flat = torch.masked_select(output.squeeze(), mask)
        # print(output)
        # print(output.shape, mask.shape, out_flat.shape)
        # break
        # out_flat = torch.masked_select(output[:,:,1], mask)
        

        label_flat = torch.masked_select(label, mask)
        # print('output',output.shape, 'mask', mask.shape,'out_flat', out_flat.shape, 'label', label.shape, 'label_flat', label_flat.shape)
        loss = criterion(out_flat, label_flat)
        
        # print(out_flat, label_flat)
        
        # record loss
        
        output_np = out_flat.detach().cpu().numpy().tolist()
        
        target_np = label_flat.detach().cpu().numpy().tolist()
        
        # print(output_np, target_np)
        
        losses.update(loss.item(), data.size(0))
        
        output_list += output_np
        target_list += target_np

        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    step_mae = mean_absolute_error(target_list, output_list)

    lr_scheduler.step(step_mae)

    return losses.avg, output_list, target_list


def validate(val_loader, model, criterion, args):
    losses = AverageMeter('Loss', ':.4e')
    # top1 = AverageMeter('Acc@1', ':6.2f')
    # met_roc = AverageMeter('ROC', ':6.2f')
    # met_pr = AverageMeter('PR', ':6.2f')
    
    output_list = []
    target_list = [] 
    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label, mask) in enumerate(val_loader):
            if args['gpu'] is not None:
                data = data.cuda(args['gpu'], non_blocking=True).type(torch.float)
                data = data.view(data.size(0), 1, data.size(2), data.size(1))
            if torch.cuda.is_available():
                label = label.cuda(args['gpu'], non_blocking=True).type(torch.float) ### although its a regression value but its called label following the source code from classification
                mask  = mask.cuda(args['gpu'], non_blocking=True).type(torch.bool)

            # compute output
            output = model(data)
            
            out_flat = torch.masked_select(output.squeeze(), mask)
            label_flat = torch.masked_select(label, mask)

            loss = criterion(out_flat, label_flat)
            
            output_np = out_flat.detach().cpu().numpy().tolist()
            target_np = label_flat.detach().cpu().numpy().tolist()

            # measure accuracy and record loss
            # acc1 = accuracy(out_flat, label_flat, topk=(1, 1))
            losses.update(loss.item(), data.size(0))
            # top1.update(acc1[0], data.size(0))
            
            output_list += output_np
            target_list += target_np
            
            # met_roc.update(roc, data.size(0))
            # met_pr.update(pr, data.size(0))

    return losses.avg, output_list, target_list

def load_model(model_path, model_class, *model_args, **model_kwargs):
    """
    Load the model from a saved state dictionary.

    Parameters:
    - model_path: Path to the saved model state dictionary.
    - model_class: The class of the model to instantiate.
    - model_args: Positional arguments for the model class instantiation.
    - model_kwargs: Keyword arguments for the model class instantiation.

    Returns:
    - model: The loaded model ready for prediction.
    """
    # Instantiate the model
    model = model_class(*model_args, **model_kwargs)
    # Load the saved state dictionary
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode
    return model

# def predict(test_loader, model, criterion, args):
#     """
#     Predict the labels for the given data using the loaded model.

#     Parameters:
#     - model: The loaded model ready for prediction.
#     - data_loader: DataLoader for the dataset to predict on.

#     Returns:
#     - predictions: Predictions for the input data.
#     """
#     predictions = []
#     with torch.no_grad():  # No need to track gradients
#         for data in test_loader:
#             # Assuming your model expects data in a specific format, adjust accordingly
#             if torch.cuda.is_available():
#                 data = data.to('cuda')
#             output = model(data)
#             # Convert output to probabilities and then to the desired format
#             # For example, using softmax for classification
#             prob = torch.softmax(output, dim=1)
#             predicted_classes = prob.argmax(dim=1)
#             predictions.append(predicted_classes.cpu().numpy())  # Move to CPU and convert to numpy if needed
#     return predictions

In [ ]:
main_work(args)

Models will be saved in: saved_models/Kidney_Reg_attentionMSE_04.09 02:43
Use GPU: 0 for training
features / nodes 231 total time graphs 32 time series length 2016 classes 1
Using cudnn.benchmark.
****************************************************
Dataset:  Kidney_Reg_attentionMSE


  0%|                                                    | 0/20 [00:00<?, ?it/s]

TRAIN, epoch 0, train_loss 5.26772, train_MAE 1.69481, train_MSE 5.70603, train_R2S -1.02435
VAL, epoch 0, val_loss 4.88847, val_MAE 1.50550, val_MSE 5.17434, val_R2S -0.73091


  5%|██                                       | 1/20 [04:13<1:20:07, 253.03s/it]

TEST, epoch 0, test_loss 4.13319, test_MAE 1.41514, test_MSE 4.63168, test_R2S -0.75068
